In [1]:
file='./jj.csv'

In [2]:
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
#from tqdm import tqdm_notebook
from tqdm.notebook import tqdm
import tqdm as tqdm
import numpy as np
import pandas as pd
from itertools import product

In [3]:
data = pd.read_csv(file)

In [4]:
print(data)

          date   data
0   1960-01-01   0.71
1   1960-04-01   0.63
2   1960-07-02   0.85
3   1960-10-01   0.44
4   1961-01-01   0.61
..         ...    ...
79  1979-10-01   9.99
80  1980-01-01  16.20
81  1980-04-01  14.67
82  1980-07-02  16.02
83  1980-10-01  11.61

[84 rows x 2 columns]


In [ ]:
plt.figure(figsize=[15, 7.5]); # Set dimensions for figure
plt.plot(data['date'], data['data'])
plt.title('Quarterly EPS for Johnson & Johnson')
plt.ylabel('EPS per share ($)')
plt.xlabel('Date')
plt.xticks(rotation=90)
plt.grid(True)
plt.show()

In [ ]:
plot_pacf(data['data']);
plot_acf(data['data']);

ACF gradually decreasing 
PACF cuts after x lag. 
Suggests AR (x) 


ACF cuts after x lag  
PACF gradual decresaing 
MA(x)



In [ ]:
ad_fuller_result = adfuller(data['data'])
alpha=0.05
print("======= ADF raw results=====")
print(ad_fuller_result)
print("======= ADF raw results=====")
adf_statistic_result = adfuller(data['data'])
adf_statistic=ad_fuller_result[0]
p_value=ad_fuller_result[1]
print(f'ADF Statistic: {adf_statistic}')
print(f'p-value: {p_value}')
if (p_value < alpha):
    print(f'series is stationary p_value={p_value}')
else:
    print(f'series is NOT stationary p_value={p_value}')

In [ ]:
data.columns

In [ ]:
data_diff_series=data['data'].diff(1)

In [ ]:
data_diff_df=pd.concat([data['date'], data_diff_series], axis=1).reset_index()

In [ ]:
data_diff_df


In [ ]:
data_diff_df=data_diff_df.dropna()


In [ ]:
data_diff_df

In [ ]:
plt.figure(figsize=[15, 7.5]); # Set dimensions for figure
plt.plot(data_diff_df['date'], data_diff_df['data'])
plt.title('Quarterly EPS for Johnson & Johnson')
plt.ylabel('EPS per share ($)')
plt.xlabel('Date')
plt.xticks(rotation=90)
plt.grid(True)
plt.show()

In [ ]:
ad_fuller_result = adfuller(data_diff_df['data'])
alpha=0.05
print("======= ADF raw results=====")
print(ad_fuller_result)
print("======= ADF raw results=====")
adf_statistic=ad_fuller_result[0]
p_value=ad_fuller_result[1]
print(f'ADF Statistic: {adf_statistic}')
print(f'p-value: {p_value}')
if (p_value < alpha):
    print(f'series is stationary p_value={p_value}')
else:
    print(f'series is NOT stationary p_value={p_value}')

In [ ]:
ad_fuller_result[1]

Still not stationary therefore take the log - looks like increaseing sd over sample

In [ ]:
#data_diff_series = np.log(data_diff_df['data'])
data_log_series = np.log(data['data'])
data_diff_and_log_df=pd.concat([data['date'], data_log_series], axis=1).reset_index()


In [ ]:
plt.figure(figsize=[15, 7.5]); # Set dimensions for figure
plt.plot(data_diff_and_log_df['date'], data_diff_and_log_df['data'])
plt.title('Quarterly EPS for Johnson & Johnson')
plt.ylabel('EPS per share ($)')
plt.xlabel('Date')
plt.xticks(rotation=90)
plt.grid(True)
plt.show()

Amplitude compressed using log but still increasing

In [ ]:
#Data querterly therefore take the first from forth to give stationary series
data_log_diff_series=data_log_series.diff(4)
data_log_diff_df=pd.concat([data['date'], data_log_diff_series], axis=1).reset_index()
data_log_diff_df=data_log_diff_df.dropna()

In [ ]:
data_log_diff_df

In [ ]:
plt.figure(figsize=[15, 7.5]); # Set dimensions for figure
plt.plot(data_log_diff_df['date'], data_log_diff_df['data'])
plt.title('Quarterly EPS for Johnson & Johnson')
plt.ylabel('EPS per share ($)')
plt.xlabel('Date')
plt.xticks(rotation=90)
plt.grid(True)
plt.show()

In [ ]:
ad_fuller_result = adfuller(data_log_diff_df['data'])
alpha=0.05
print("======= ADF raw results=====")
print(ad_fuller_result)
print("======= ADF raw results=====")
adf_statistic=ad_fuller_result[0]
p_value=ad_fuller_result[1]
print(f'ADF Statistic: {adf_statistic}')
print(f'p-value: {p_value}')
if (p_value < alpha):
    print(f'series is stationary p_value={p_value}')
else:
    print(f'series is NOT stationary p_value={p_value}')

In [ ]:
plot_pacf(data_log_diff_df['data']);
plot_acf(data_log_diff_df['data']);

PACF that we have a significant peak at lag 1
suggests an AR(1)

Another peak at lag 4, 
seasonal autoregressive process of order 1 (P = 1).

ACF plot significant peak at lag 1
suggesting a non-seasonal MA(1) process.

However use multiple parameters and AIC to measure

In [ ]:
def optimize_SARIMA(parameters_list, d, D, s, data):
    """
        Return dataframe with parameters, corresponding AIC and SSE
        
        parameters_list - list with (p, q, P, Q) tuples
        d - integration order
        D - seasonal integration order
        s - length of season
        exog - the exogenous variable
    """
    
    results = []
    
    #for param in tqdm.notebook.tqdm(parameters_list):
    for param in parameters_list:
        try: 
            model = SARIMAX(data, order=(param[0], d, param[1]), seasonal_order=(param[2], D, param[3], s)).fit(disp=-1)
        except:
            continue
            
        aic = model.aic
        results.append([param, aic])
        
    result_df = pd.DataFrame(results)
    result_df.columns = ['(p,q)x(P,Q)', 'AIC']
    #Sort in ascending order, lower AIC is better
    result_df = result_df.sort_values(by='AIC', ascending=True).reset_index(drop=True)
    
    return result_df

In [ ]:
p = range(0, 4, 1)
d = 1
q = range(0, 4, 1)
P = range(0, 4, 1)
D = 1
Q = range(0, 4, 1)
s = 4
parameters = product(p, q, P, Q)
parameters_list = list(parameters)
print(len(parameters_list))
parameters = product(p, q, P, Q)
parameters_list = list(parameters)
print(len(parameters_list))

In [ ]:
result_df=optimize_SARIMA(parameters_list, d, D, s, data['data'])
print(result_df)

In [ ]:
print(result_df)
result_df[result_df['AIC'] == result_df['AIC'].min()]

In [ ]:
best_model = SARIMAX(data['data'], order=(2, d, 3), seasonal_order=(1, D, 0, 4)).fit(dis=-1)
print(best_model.summary())

In [ ]:
data['arima_model'] = best_model.fittedvalues
data['arima_model'][:4+1] = np.NaN
forecast = best_model.predict(start=data.shape[0], end=data.shape[0] + 8)
forecast = data['arima_model'].append(forecast)
plt.figure(figsize=(15, 7.5))
plt.plot(forecast, color='r', label='model')
plt.axvspan(data.index[-1], forecast.index[-1], alpha=0.5, color='lightgrey')
plt.plot(data['data'], label='actual')
plt.legend()
plt.show()